# Tag items including faces #
This notebook analyzes Tropy item photos for faces. If it finds them, it tags the Tropy item `People`

In [1]:
import cv2
import tropy

In [2]:
# The tropy module defaults to http://localhost:2019; uncomment and set address/port here only if necessary 
# tropy.api_url_base = "http://host:port"

In [3]:
# Configure to look for faces

cascPath = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"

# Create the haar cascade

faceCascade = cv2.CascadeClassifier(cascPath)

In [4]:
# Analyze an image for faces and count the results
# (drawing on may examples out there, e.g. https://realpython.com/face-recognition-with-python/) 

def count_faces(imagePath):
    # Read the image
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )

    return len(faces)

In [5]:
# Get the images a from each Tropy item in list of items
# Since we are only interested in groups of people, we'll set a threshold of faces to ignore false positives
# If item has any photo above the threshold, tag item as "People"

items = tropy.get_all_items()
threshold = 5
tags = ['People']

for item in items:
    photos = tropy.get_item_photos(item)
    for photo in photos:
        faces = count_faces(photo)
        if faces > threshold:
            tropy.tag_item_by_tag_name(item, tags)
            break